In [5]:
def gen_title(settings, user_input):
    response = openai.chat.completions.create(
        model= settings["model"],
        messages= [
            {
                "role": "system",
                "content": 
                    f"""
                    You are an expert historian that specializes in building book projects.
                    Your task is to propose compelling titles for a book based on the provided subject.
                    The tone of voice for the book is: {settings['tone_of_voice']}.
                    """
            },            
            {
                "role": "system",
                "content": 
                    """
                    The output is a JSON document like:

                    {
                      "titles": [
                        {
                          "id": 1,
                          "title": "Title 1...",
                          "subtitle": "Subtitle 1..."
                        },
                        {
                          "id": 2,
                          "title": "Title 2...",
                          "subtitle": "Subtitle 2..."
                        }
                      ]
                    }
                    """
            },
            {
                "role": "system",
                "content": 
                    """
                    The both title and subtitle should be simple sentences.
                    Avoid punctuation that suggests co-dependence like ":" or ".".
                    The title should be short and catchy.
                    The subtitle should provide info about the subject that pushes the user to read the content.
                    """
            },
            {
                "role": "user",
                "content": f"Generate between {settings['titles_min']} and {settings['titles_min']} good titles for a book on the subject: {settings['subject']}"
            }
        ]
    )
    data = text2json(response.choices[0].message.content)
    return data

In [6]:
def prompt_title(titles):
    """
    Prompts the user to select a title from a list of titles.

    Parameters:
    titles (list): List of dictionaries containing 'id', 'title', and 'subtitle' keys.

    Returns:
    dict: The selected title dictionary.
    """
    # ANSI escape codes for formatting
    bold = "\033[1m"
    reset = "\033[0m"
    blue = "\033[94m"

    # Display the options to the user
    print("Please choose a title from the following options:")
    for i, title in enumerate(titles, 1):
        id_str = f"{i}." if i >= 10 else f" {i}."
        print(f"{id_str} {bold}{title['title']}{reset}")
        print(f"    {blue}{title['subtitle']}{reset}\n")  # Add color to the subtitle

    # Get user input and validate it
    while True:
        try:
            choice = input("Enter the number of your choice: ").strip()
            if choice.isdigit():
                choice = int(choice)
                if 1 <= choice <= len(titles):
                    return titles[choice - 1]
                else:
                    print(f"Please enter a number between 1 and {len(titles)}.")
            else:
                print("Invalid input. Please enter a number.")
        except ValueError:
            print("Invalid input. Please enter a number.")